# Proyecto Final  

Pablo Izquierdo Conde

Jorge Pose Eiroa

In [1]:
import wikipediaapi as wk

wiki = wk.Wikipedia('en')

# Cogemos las categorías seleccionadas
cat1 = wiki.page("Category:Esports")
cat2 = wiki.page("Category:Combat_sports")

In [30]:
# Función que busca recursivamente en las categorías y nos devuelve una lista con las páginas
def print_categorymembers(categorymembers, level=0, max_level=2, pages = []):
    for c in categorymembers.values():
        if c.ns == 0: # Sacamos las páginas (ns=0)
            pages.append(c)
            #print("%s: %s (ns: %d)" % ("*" * (level + 1), c.title, c.ns))
        if c.ns == wk.Namespace.CATEGORY and level <= max_level:
            print_categorymembers(c.categorymembers, level + 1,1, pages)
p1 = []
p2 = []
print_categorymembers(cat1.categorymembers,0,1, p1)
print_categorymembers(cat2.categorymembers,0,1, p2)

*: Esports (ns: 0)
*: College esports (ns: 0)
*: Gaming house (ns: 0)
*: LAN party (ns: 0)
*: XQc (ns: 0)
*: Meltdown (bar chain) (ns: 0)
*: Oceanic Pro League (ns: 0)
*: Twitch gameplay (ns: 0)
***: ESL Pro League Season 4 (ns: 0)
***: Canada Cup (fighting game event) (ns: 0)
***: HUD (TV program) (ns: 0)
***: Nintendo Campus Challenge (ns: 0)
***: World Cyber Games 2009 (ns: 0)
***: Thailand E-Sports Federation (ns: 0)
***: World Cyber Games 2008 (ns: 0)
***: World Cyber Games 2006 (ns: 0)
***: ESports National Association of the Philippines (ns: 0)
***: Russian eSports Federation (ns: 0)
***: World Cyber Games 2005 (ns: 0)
***: International Esports Federation (ns: 0)
***: Korea e-Sports Association (ns: 0)
***: World Cyber Games 2001 (ns: 0)
***: World Cyber Games 2002 (ns: 0)
***: World Cyber Games 2003 (ns: 0)
***: World Cyber Games 2011 (ns: 0)
***: World Cyber Games Challenge (ns: 0)
***: Donkey Kong high score competition (ns: 0)
***: Professional eSports Association (ns: 0)
*

***: Traje de luces (ns: 0)
***: Unfortunate events in the front seats of the ring of Madrid, and the death of the mayor of Torrejón (ns: 0)
***: Vaquejada (ns: 0)
***: Verónica (bullfighting) (ns: 0)
***: Cockfight (ns: 0)
***: Aadukalam (ns: 0)
***: The Cock Fight (ns: 0)
***: Cock Magic (ns: 0)
***: Cockfighter (ns: 0)
***: Deep Play: Notes on the Balinese Cockfight (ns: 0)
***: The Rum Diary (film) (ns: 0)
***: Town Police Clauses Act 1847 (ns: 0)
***: Gladiator (ns: 0)
***: Arbelas (ns: 0)
***: Ave Imperator, morituri te salutant (ns: 0)
***: Damnatio ad bestias (ns: 0)
***: De spectaculis (ns: 0)
***: Dimachaerus (ns: 0)
***: Gladiator Mosaic (ns: 0)
***: Gladiatrix (ns: 0)
***: Manica (armguard) (ns: 0)
***: Naumachia (ns: 0)
***: Pollice verso (ns: 0)
***: Corrie Coursing Meeting (ns: 0)
***: Waterloo Cup (ns: 0)
***: Hunting (ns: 0)
***: Alanorarius (ns: 0)
***: Trapping (ns: 0)
***: Bag (fishing and hunting) (ns: 0)
***: Bag limits (ns: 0)
***: Bambi effect (ns: 0)
***: Bay d

***: Brazilian National Jiu-Jitsu Championship (ns: 0)
***: Brazilian Nationals Jiu-Jitsu No-Gi Championship (ns: 0)
***: Eddie Bravo Invitational (ns: 0)
***: European Championship (Brazilian jiu-jitsu) (ns: 0)
***: European Nogi Brazilian Jiu Jitsu Championship (ns: 0)
***: Grapplers Quest (ns: 0)
***: International Brazilian Jiu-Jitsu Federation (ns: 0)
***: Masahiko Kimura vs. Hélio Gracie (ns: 0)
***: Master and seniors international jiu-jitsu championship (ns: 0)
***: Metamoris (ns: 0)
***: North American Grappling Association (ns: 0)
***: Pan Jiu-Jitsu No-Gi Championship (ns: 0)
***: Pan-American Championship (jiu-jitsu) (ns: 0)
***: Polaris Pro Grappling (ns: 0)
***: Rio International Open Jiu-Jitsu Championship (ns: 0)
***: SJJIF Worlds Gi & NoGi Tournament(NABJJ) (ns: 0)
***: World Jiu-Jitsu Championship (ns: 0)
***: World Nogi Brazilian Jiu-Jitsu Championship (ns: 0)
***: World Professional Jiu-Jitsu Cup (ns: 0)
***: World Professional Jiu-Jitsu Cup 2011 (ns: 0)
***: 10th Pl

***: Naihanchi (ns: 0)
***: Nijūshiho (ns: 0)
***: Passai (ns: 0)
***: Pinan (ns: 0)
***: Rōhai (ns: 0)
***: Sanchin (ns: 0)
***: Seisan (ns: 0)
***: Seiunchin (ns: 0)
***: Shisochin (ns: 0)
***: Sōchin (ns: 0)
***: Taikyoku (ns: 0)
***: Tensho (kata) (ns: 0)
***: Unsū (ns: 0)
***: Wankan (ns: 0)
***: Wanshū (ns: 0)
***: Asian Karatedo Federation (ns: 0)
***: Australian Karate Federation (ns: 0)
***: Deutscher JKA-Karate Bund (ns: 0)
***: Egypt Karate Federation (ns: 0)
***: European Karate Federation (ns: 0)
***: Japan Karate Federation (ns: 0)
***: French Karate Federation (ns: 0)
***: Genseiryū Karate-do International Federation (ns: 0)
***: German Karate Federation (ns: 0)
***: Ghana Karate Do Association (ns: 0)
***: Hellenic Karate Federation (ns: 0)
***: International Karate Organization (ns: 0)
***: International Okinawan Gōjū-ryū Karate-dō Federation (ns: 0)
***: International Shotokan Karate Federation (ns: 0)
***: International Sport Karate Association (ns: 0)
***: Internati

***: Fight Master: Bellator MMA (ns: 0)
***: Hard Knocks Fighting (ns: 0)
***: Inside MMA (ns: 0)
***: Iron Ring (TV series) (ns: 0)
***: MMA Live (ns: 0)
***: ShoMMA (ns: 0)
***: ShoXC (ns: 0)
***: Strikeforce (mixed martial arts) (ns: 0)
***: Tapout (TV series) (ns: 0)
***: Warrior Xtreme Cagefighting (ns: 0)
***: Wimp 2 Warrior (ns: 0)
***: World Combat League (ns: 0)
***: Kunlun Fight World Combat Sports Center (ns: 0)
***: Mixed martial arts weight classes (ns: 0)
***: Atomweight (MMA) (ns: 0)
***: Bantamweight (MMA) (ns: 0)
***: Catchweight (ns: 0)
***: Cruiserweight (MMA) (ns: 0)
***: Featherweight (MMA) (ns: 0)
***: Flyweight (MMA) (ns: 0)
***: Heavyweight (MMA) (ns: 0)
***: Light heavyweight (MMA) (ns: 0)
***: Lightweight (MMA) (ns: 0)
***: Middleweight (MMA) (ns: 0)
***: Openweight (ns: 0)
***: Strawweight (MMA) (ns: 0)
***: Super heavyweight (MMA) (ns: 0)
***: Super lightweight (MMA) (ns: 0)
***: Super middleweight (MMA) (ns: 0)
***: Super welterweight (MMA) (ns: 0)
***: Wel

***: Heavyweight (MMA) (ns: 0)
***: Light heavyweight (MMA) (ns: 0)
***: Lightweight (MMA) (ns: 0)
***: Middleweight (MMA) (ns: 0)
***: Openweight (ns: 0)
***: Strawweight (MMA) (ns: 0)
***: Super heavyweight (MMA) (ns: 0)
***: Super lightweight (MMA) (ns: 0)
***: Super middleweight (MMA) (ns: 0)
***: Super welterweight (MMA) (ns: 0)
***: Welterweight (MMA) (ns: 0)
***: Professional wrestling weight classes (ns: 0)
***: Cruiserweight (professional wrestling) (ns: 0)
***: Featherweight (ns: 0)
***: Flyweight (ns: 0)
***: Heavyweight (ns: 0)
***: Middleweight (ns: 0)
***: Mini flyweight (ns: 0)
***: Taekwondo weight classes (ns: 0)
***: Welterweight (ns: 0)
***: Bantamweight (ns: 0)
***: Featherweight (ns: 0)
***: Flyweight (ns: 0)
***: Heavyweight (ns: 0)
***: Lightweight (ns: 0)
***: Middleweight (ns: 0)
***: Super heavyweight (ns: 0)
***: Welterweight (ns: 0)
***: Wrestling weight classes (ns: 0)
**: Aikido World Championships (ns: 0)
**: FILA World Grappling Championship (ns: 0)
**: 

In [31]:
print(len(p1))
print(len(p2))

1224
4288


In [42]:
print(p2[0].text)

"Combat Sports" redirects here, for The Vaccines' album, see Combat Sports (album).A combat sport, or fighting sport, is a competitive contact sport that usually involves one-on-one combat. In many combat sports, a contestant wins by scoring more points than the opponent or by disabling the opponent. Common combat sports include mixed martial arts, boxing, wrestling, fencing, savate, kickboxing, Muay Thai, Sanda, Tae Kwon Do, Capoeira, Brazilian Jiu-jitsu, HMB, Sambo, Kyokushin, and Kūdō.

History
Tradition styles of wrestling exist in most cultures; wrestling can be considered a cultural universal. Boxing contests date back to ancient Sumer in the 3rd millennium BCE and ancient Egypt circa 1350 BCE. The ancient Olympic Games included several combat-related sports: armored foot races, boxing, wrestling, and pankration, which was introduced in the Olympic Games of 648 BCE.
In ancient China, combat sport appeared in the form of lei tai. It was a no-holds barred combat sport that combined

# 2. Procesado del texto

Una vez obtenidas los documentos, hacemos el procesado del texto y el modelado de tópicos

El procesado consta de las siguientes partes:

1. Tokenization
2. Homogeneization
3. Cleaning
4. Vectorization

In [56]:
# Sacamos el texto de las listas de páginas

corpus0 = []
corpus1 = []

corpus0 = [p.text for p in p1]
corpus1 = [p.text for p in p2]

KeyboardInterrupt: 

In [53]:
print(len(corpus0))
print(len(corpus1))

Heroes of Newerth (HoN) is a multiplayer online battle arena (MOBA) video game originally developed by S2 Games for Microsoft Windows, macOS, and Linux. The game idea was derived from the Warcraft III: The Frozen Throne custom map Defense of the Ancients and was S2 Games' first MOBA title. The game was released on May 12, 2010, and re-released as a free-to-play game on July 29, 2011. On May 5, 2015, Heroes of Newerth development duties passed to Frostburn Studios, with the development team moving over to the new company.

Gameplay
General
Heroes of Newerth pits two teams of players against each other: the Legion and the Hellbourne. Both teams are based at opposite corners of the map in their respective bases. Bases consist of buildings, barracks, towers, a hero spawning pool, and a central structure. The goal of the game is to either destroy the central structure, the World Tree (Legion) or Sacrificial Shrine (Hellbourne), of the opposite base or force the other team to concede. Player